In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
import re
import string
import tensorflow as tf
import tensorflow_datasets as tfds
import transformers
from transformers import TFAutoModel, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
from nltk.corpus import stopwords
import warnings
import ast
warnings.filterwarnings("ignore")

In [3]:
tutor_df = pd.read_csv('/kaggle/input/dataset-tutor-with-embedding/dataset_tutor_vector.csv')

In [4]:
# Convert the string representations of embeddings back to numerical arrays
tutor_df['Tokenized_Kriteria'] = tutor_df['Tokenized_Kriteria'].apply(ast.literal_eval)

In [5]:
tutor_df.drop(columns=['Unnamed: 0'], inplace=True)

In [6]:
tutor_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TutorID              432 non-null    int64  
 1   Jenis Kelamin        432 non-null    object 
 2   Pendidikan Terakhir  432 non-null    object 
 3   Provinsi             432 non-null    object 
 4   Kabupaten/Kota       432 non-null    object 
 5   Interest             432 non-null    object 
 6   Bahasa               432 non-null    object 
 7   Kriteria             432 non-null    object 
 8   Skor Label           364 non-null    float64
 9   Label                112 non-null    float64
 10  Tokenized_Kriteria   432 non-null    object 
dtypes: float64(2), int64(1), object(8)
memory usage: 37.2+ KB


In [6]:
tutor_df["Jenis Kelamin"].unique()

array(['Laki-laki', 'Perempuan'], dtype=object)

In [7]:
tutor_df["Pendidikan Terakhir"].unique()

array(['S1', 'S3', 'S2', 'SMA', 'S1 Teknik Informatika', 'S1 Psikologi',
       'S2 Manajemen', 'S1 Pendidikan Matematika', 'S1 Teknik Elektro',
       'S1 Sastra Inggris', 'S1 Ilmu Komputer', 'S1 Hukum',
       'S1 Teknik Sipil', 'S1 Akuntansi', 'S1 Kedokteran', 'S1 Biologi',
       'S1 Seni Rupa', 'S1 Pendidikan Fisika', 'S1 Teknik Kimia',
       'S1 Ilmu Komunikasi', 'S1 Pendidikan Bahasa Inggris', 'S1 Farmasi',
       'S1 Teknik Industri', 'S1 Sastra Jepang',
       'S1 Pendidikan Bahasa Indonesia'], dtype=object)

In [8]:
tutor_df["Provinsi"].unique()

array(['Daerah Istimewa Yogyakarta', 'Jakarta ', 'Jawa Barat', 'Jakarta',
       'Banten', 'DKI Jakarta', 'Jawa Timur', 'Jawa Tengah',
       'Sumatera Utara', 'Bali', 'Sulawesi Selatan', 'Kalimantan Timur',
       'Sumatera Selatan', 'Kalimantan Selatan', 'Kepulauan Riau',
       'Sumatera Barat', 'Kalimantan Barat', 'Sulawesi Utara', 'Lampung',
       'Maluku', 'Sulawesi Tenggara', 'Nusa Tenggara Barat', 'Papua',
       'Kalimantan Utara', 'Sulawesi Tengah', 'Aceh', 'Maluku Utara',
       'Riau'], dtype=object)

In [9]:
tutor_df["Kabupaten/Kota"].unique()

array(['Sleman', 'Sleman ', 'Jakarta barat', 'Jakarta timur ', 'Bogor',
       'Jakarta Timur', 'Jakarta Barat', 'Cilegon', 'Jakarta Pusat',
       'Kabupaten Bekasi', 'Jakarta Selatan', 'Bandung', 'Surabaya',
       'Yogyakarta', 'Semarang', 'Malang', 'Bekasi', 'Medan', 'Denpasar',
       'Jakarta Utara', 'Makassar', 'Balikpapan', 'Palembang',
       'Banjarmasin', 'Depok', 'Surakarta', 'Tangerang', 'Batam',
       'Padang', 'Pontianak', 'Manado', 'Solo', 'Magelang',
       'Bandar Lampung', 'Ambon', 'Kendari', 'Mataram', 'Jayapura',
       'Samarinda', 'Palopo', 'Tanjung Selor', 'Palu', 'Salatiga',
       'Banda Aceh', 'Blitar', 'Ternate', 'Pekanbaru'], dtype=object)

In [10]:
tutor_df["Interest"].unique()

array(['Kalkulus, Fisika', 'Fisika ', 'Fisika', 'Hukum', 'sosiologi',
       'Machine learning', 'Data Science ', 'Matematika',
       'Pemrograman Dasar',
       'Artificial Intelligence, spesialisasi Prompt Engineering',
       'Sejarah', 'Data Analyst', 'Pemrograman', 'Bahasa Inggris',
       'Data Science', 'Data Analysis', 'Kimia', 'Ekonomi', 'Geografi',
       'Desain Grafis', 'Fotografi', 'Bahasa Jepang', 'Biologi',
       'Ilmu Komputer', 'Public Speaking', 'Seni Lukis', 'Bisnis Digital',
       'Machine Learning', 'Web Development', 'Digital Marketing',
       'Artificial Intelligence', 'Deep Learning', 'Computer Vision',
       'DevOps', 'Natural Language Processing', 'SEO', 'Digital Art',
       'Cloud Computing', 'Photography', 'Economics', 'Graphic Design',
       'UX/UI Design', 'Cybersecurity', 'Blockchain',
       'Matematika dan Fisika', 'Biologi dan Kimia', 'Big Data',
       'Game Development', 'Bisnis Kuliner', 'Pemasaran Digital',
       'Bisnis Fashion', 'Bisnis P

In [11]:
tutor_df["Bahasa"].unique()

array(['Indonesia', 'Inggris', 'Indonesia dan Inggris',
       'Indonesia, Inggris', 'Indonesia, Jepang'], dtype=object)

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
from transformers import TFAutoModel, AutoTokenizer, DataCollatorWithPadding, TFBertModel
import logging
import datetime
import tensorflow as tf
import keras
import string
import re

In [13]:
# tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
# bert = TFAutoModel.from_pretrained("indobenchmark/indobert-base-p1")

In [14]:
# def tokenize_kriteria(kriteria_list):
#     # Tokenize the list of kriteria
#     inputs = tokenizer(
#         kriteria_list,
#         max_length=128,           # Example max length
#         truncation=True,          # Truncate to max_length
#         padding='max_length',     # Pad to max_length
#         return_tensors='tf'       # Return TensorFlow tensors
#     )
#     attention = inputs['attention_mask']
#     outputs = bert(inputs)
#     embeddings = outputs[0]
#     mask = tf.cast(tf.expand_dims(attention, -1), dtype=tf.float32)
#     masked_embeddings = embeddings * mask
#     summed = tf.reduce_sum(masked_embeddings, axis=1)
#     counts = tf.clip_by_value(tf.reduce_sum(mask, axis=1), clip_value_min=1e-9, clip_value_max=tf.float32.max)
#     mean_pooled = summed / counts
#     return mean_pooled

# # Apply the tokenizer to the 'Kriteria' column
# kriteria_list = tutor_df['Kriteria'].tolist()
# tokenized_kriteria = tokenize_kriteria(kriteria_list)

# # Convert the tokenized outputs to lists
# embeddings = tokenized_kriteria.numpy().tolist()

# # Add the tokenized texts as a new column to the DataFrame
# tutor_df['Tokenized_Kriteria'] = embeddings

In [15]:
tutor_df.head()

,TutorID,Jenis Kelamin,Pendidikan Terakhir,Provinsi,Kabupaten/Kota,Interest,Bahasa,Kriteria,Skor Label,Label,Tokenized_Kriteria
0,1,Laki-laki,S1,Daerah Istimewa Yogyakarta,Sleman,"Kalkulus, Fisika",Indonesia,Saya mahasiswa fisika Universitas Gadjah Mada ...,5.0,1.0,"[0.14578990638256073, 0.8595218062400818, -0.1..."
1,2,Laki-laki,S1,Daerah Istimewa Yogyakarta,Sleman,Fisika,Indonesia,Saya berpengalaman dalam mengajar fisika dan m...,4.0,1.0,"[-0.33497920632362366, 0.6025481820106506, 0.6..."
2,3,Perempuan,S1,Daerah Istimewa Yogyakarta,Sleman,Fisika,Indonesia,Saya menggunakan beberapa warna dalam mencatat...,0.0,0.0,"[0.4659246802330017, 0.22397109866142273, -0.4..."
3,4,Perempuan,S1,Daerah Istimewa Yogyakarta,Sleman,Hukum,Indonesia,Saya sedang menekuni kuliah sarjana strata 1 h...,4.0,1.0,"[-0.267794132232666, 0.3876529335975647, 0.648..."
4,5,Laki-laki,S1,Daerah Istimewa Yogyakarta,Sleman,sosiologi,Indonesia,saya sangat mendalami pelajaran sosiologi saat...,2.0,0.0,"[0.2914064824581146, 0.48906809091567993, 0.64..."


In [21]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import re
import string

class TokenSimilarityTFLite:
    max_length = 128

    def __init__(self, truncation=True, padding="max_length", max_length=max_length):
        self.max_length = max_length
        self.truncation = truncation
        self.padding = padding
        self.tokenizer = None
        self.interpreter = None

    def load_pretrained(self, from_pretrained="indobenchmark/indobert-base-p1"):
        self.tokenizer = AutoTokenizer.from_pretrained(from_pretrained)

        # Initialize TFLite interpreter
        self.interpreter = tf.lite.Interpreter(model_path='/kaggle/input/tflite-bert/tflite/tflite/1/model.tflite')
        self.interpreter.allocate_tensors()

        # Get input and output details
        self.input_details = self.interpreter.get_input_details()
        self.output_details = self.interpreter.get_output_details()

    def cleaning(self, text: str):
        text = text.translate(str.maketrans('', '', string.punctuation))
        text = re.sub(r'\s+', ' ', text).strip()
        return text

    def process(self, token: str):
        inputs = self.tokenizer([token],
                                max_length=self.max_length,
                                truncation=self.truncation,
                                padding=self.padding,
                                return_tensors='np')

        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        token_type_ids = inputs['token_type_ids']

        # Set input tensors
        self.interpreter.set_tensor(self.input_details[0]['index'], input_ids)
        self.interpreter.set_tensor(self.input_details[1]['index'], attention_mask)
        self.interpreter.set_tensor(self.input_details[2]['index'], token_type_ids)

        # Run inference
        self.interpreter.invoke()

        # Get output embeddings
        embeddings = self.interpreter.get_tensor(self.output_details[0]['index'])

        attention = inputs['attention_mask']
        mask = tf.cast(tf.expand_dims(attention, -1), dtype=tf.float32)
        masked_embeddings = embeddings * mask
        summed = tf.reduce_sum(masked_embeddings, axis=1)
        counts = tf.clip_by_value(tf.reduce_sum(mask, axis=1), clip_value_min=1e-9, clip_value_max=tf.float32.max)
        mean_pooled = summed / counts
        return mean_pooled.numpy()

    def predict(self, first_token: str, second_token_embeddings: np.ndarray):
        first_token = self.cleaning(first_token)
        first_token_embedding = self.process(first_token)

        # Calculate similarity
        similarity = cosine_similarity(first_token_embedding, second_token_embeddings)

        return similarity

In [22]:
# Example usage
token_similarity = TokenSimilarityTFLite()
token_similarity.load_pretrained()

In [23]:
# Example of using predict method
first_token = "example token"
second_token_embeddings = np.random.rand(1, 768)  # Example of second_token_embeddings, should be actual embeddings
similarity_score = token_similarity.predict(first_token, second_token_embeddings)
print(f"Similarity score: {similarity_score}")

Similarity score: [[-0.02029861]]


In [33]:
# class TokenSimilarityTF:
#     max_length = 128
#     def __init__(self, truncation: bool = True, padding  = "max_length", max_length=max_length):
#         self.max_length = max_length
#         self.truncation = truncation
#         self.padding = padding

#     def load_pretrained(self, from_pretrained: str = "indobenchmark/indobert-base-p1"):
#         self.tokenizer = AutoTokenizer.from_pretrained(from_pretrained)
#         self.model = TFAutoModel.from_pretrained(from_pretrained)

#     def cleaning(self, text: str):
#         text = text.translate(str.maketrans('', '', string.punctuation))
#         text = re.sub(r'\s+', ' ', text).strip()
#         return text

#     def process(self, token: str):
#         inputs = self.tokenizer([token],
#                                 max_length=self.max_length,
#                                 truncation=self.truncation,
#                                 padding=self.padding,
#                                 return_tensors='tf')
#         attention = inputs['attention_mask']
#         outputs = self.model(inputs)
#         embeddings = outputs[0]
#         mask = tf.cast(tf.expand_dims(attention, -1), dtype=tf.float32)
#         masked_embeddings = embeddings * mask
#         summed = tf.reduce_sum(masked_embeddings, axis=1)
#         counts = tf.clip_by_value(tf.reduce_sum(mask, axis=1), clip_value_min=1e-9, clip_value_max=tf.float32.max)
#         mean_pooled = summed / counts
#         return mean_pooled.numpy()

#     def predict(self, first_token:str, second_token_embeddings:np.ndarray):
#         first_token = self.cleaning(first_token)
#         first_token_embedding = self.process(first_token)

#         # calculate similarity
#         similarity = cosine_similarity(first_token_embedding, second_token_embeddings)

#         return similarity

In [24]:
input_token = 'Saya ingin belajar fisika dengan mahasiswa fisika yang sudah berpengalaman menjadi asisten dosen dan juga di daerah Yogyakarta dengan menggunakan bahasa indonesia harus laki-laki.'

kriteria_embeddings = np.array(tutor_df['Tokenized_Kriteria'].tolist())

similarities = token_similarity.predict(input_token, kriteria_embeddings)

top_indices = np.argsort(similarities.flatten())[::-1]

# top_indices = np.argsort(similarities.flatten())[-10:][::-1]
top_10_similarities = tutor_df.iloc[top_indices]

top_10_similarities

,TutorID,Jenis Kelamin,Pendidikan Terakhir,Provinsi,Kabupaten/Kota,Interest,Bahasa,Kriteria,Skor Label,Label,Tokenized_Kriteria
0,1,Laki-laki,S1,Daerah Istimewa Yogyakarta,Sleman,"Kalkulus, Fisika",Indonesia,Saya mahasiswa fisika Universitas Gadjah Mada ...,5.0,1.0,"[0.14578990638256073, 0.8595218062400818, -0.1..."
6,7,Laki-laki,S1,Jakarta,Jakarta timur,Machine learning,Inggris,Saya seseorang yang berpengalaman dalam bidang...,5.0,1.0,"[0.45113304257392883, 0.563999354839325, 0.087..."
15,16,Perempuan,S2,Jawa Barat,Kabupaten Bekasi,Fisika,Indonesia,Saya punya pengalaman dalam mengajar fisika da...,5.0,1.0,"[0.5918110609054565, 0.06050441041588783, 0.02..."
3,4,Perempuan,S1,Daerah Istimewa Yogyakarta,Sleman,Hukum,Indonesia,Saya sedang menekuni kuliah sarjana strata 1 h...,4.0,1.0,"[-0.267794132232666, 0.3876529335975647, 0.648..."
9,10,Laki-laki,S1,Jakarta,Jakarta Timur,Matematika,Indonesia,Sangat menguasai matematik dan memiliki pengal...,5.0,1.0,"[0.13022500276565552, 0.32530316710472107, -0...."
...,...,...,...,...,...,...,...,...,...,...,...
342,333,Laki-laki,S3,Sumatera Selatan,Palembang,Insurtech,Indonesia,"Berpengalaman 12 tahun dalam insurtech, ahli d...",4.8,NaN,"[-0.8284884691238403, 0.9281118512153625, 0.21..."
362,353,Perempuan,S2,Jawa Barat,Depok,AI in Retail,Indonesia,"Berpengalaman 8 tahun dalam ritel dan AI, foku...",4.7,NaN,"[-0.7301281094551086, 0.5445655584335327, -0.2..."
180,171,Laki-laki,SMA,Jawa Tengah,Magelang,Matematika,Indonesia dan Inggris,Menyediakan bimbingan yang terstruktur serta m...,3.0,NaN,"[-0.26058605313301086, 0.7267490029335022, -0...."
194,185,Laki-laki,S2,Jawa Tengah,Solo,Matematika,Indonesia dan Inggris,Menyediakan bimbingan yang terstruktur serta m...,5.0,NaN,"[-0.26058605313301086, 0.7267490029335022, -0...."


In [25]:
gender_filter = 'Laki-laki'  # or 'Perempuan'
mapel_list = ['Fisika']  # List of subjects

# Convert the list of subjects to a regular expression pattern
mapel_pattern = '|'.join(mapel_list)

filtered_df = top_10_similarities[(top_10_similarities['Jenis Kelamin'] == gender_filter) &
              (top_10_similarities['Interest'].str.contains(mapel_pattern))]

In [26]:
filtered_df

,TutorID,Jenis Kelamin,Pendidikan Terakhir,Provinsi,Kabupaten/Kota,Interest,Bahasa,Kriteria,Skor Label,Label,Tokenized_Kriteria
0,1,Laki-laki,S1,Daerah Istimewa Yogyakarta,Sleman,"Kalkulus, Fisika",Indonesia,Saya mahasiswa fisika Universitas Gadjah Mada ...,5.0,1.0,"[0.14578990638256073, 0.8595218062400818, -0.1..."
1,2,Laki-laki,S1,Daerah Istimewa Yogyakarta,Sleman,Fisika,Indonesia,Saya berpengalaman dalam mengajar fisika dan m...,4.0,1.0,"[-0.33497920632362366, 0.6025481820106506, 0.6..."
410,401,Laki-laki,S1 Pendidikan Fisika,Riau,Pekanbaru,Fisika,Indonesia,"Pendidikan minimal S1, bisa memberikan bimbing...",NaN,NaN,"[-0.19629693031311035, 0.676673948764801, 1.38..."
62,53,Laki-laki,S2,DKI Jakarta,Jakarta Pusat,Fisika,Indonesia,Mengajar fisika untuk SMA dengan pengalaman 5 ...,3.0,0.0,"[-0.3235238194465637, 0.8790085911750793, -0.0..."
47,48,Laki-laki,S2,Jawa Timur,Surabaya,Fisika,Indonesia,Mengajar fisika untuk SMA dengan pengalaman 5 ...,3.0,0.0,"[-0.3235238194465637, 0.8790085911750793, -0.0..."
415,406,Laki-laki,S1 Teknik Elektro,Sumatera Utara,Medan,"Fisika, Kimia,","Indonesia, Inggris","Menguasai Fisika dan Kimia, bisa memberikan le...",NaN,NaN,"[-0.07188328355550766, 1.0404706001281738, 0.5..."
394,385,Laki-laki,S1 Teknik Elektro,Sumatera Utara,Medan,"Fisika, Kimia","Indonesia, Inggris","Menguasai Fisika dan Kimia, bisa memberikan le...",NaN,NaN,"[-0.07188328355550766, 1.0404706001281738, 0.5..."
368,359,Laki-laki,S1 Teknik Elektro,Sumatera Utara,Medan,"Fisika, Kimia","Indonesia, Inggris","Menguasai Fisika dan Kimia, bisa memberikan le...",NaN,NaN,"[-0.07188328355550766, 1.0404706001281738, 0.5..."
411,402,Laki-laki,S1 Teknik Informatika,Jawa Barat,Bandung,"Matematika, Fisika,","Indonesia, Inggris","Pendidikan minimal S1, bisa mengajar Matematik...",NaN,NaN,"[-0.7965583801269531, 0.31829407811164856, 1.0..."
369,360,Laki-laki,S1 Teknik Informatika,Jawa Barat,Depok,"Matematika, Fisika","Indonesia, Inggris","Pendidikan minimal S1, bisa mengajar Matematik...",NaN,NaN,"[-0.7965583801269531, 0.31829407811164856, 1.0..."


In [ ]:
# Use the predict_with_fine_tuned method
token1 = "halo aku jaki"
token2 = "halo aku iko"

similarity_score = model.predict_with_fine_tuned(token1, token2)
print(f"Similarity score between '{token1}' and '{token2}': {similarity_score}")

AttributeError: 'TFBertModel' object has no attribute 'predict_with_fine_tuned'

In [ ]:
# """
# This examples trains BERT (or any other transformer model like RoBERTa, DistilBERT etc.) for the STSbenchmark from scratch. It generates sentence embeddings
# that can be compared using cosine-similarity to measure the similarity.

# Usage:
# python training_nli.py

# OR
# python training_nli.py pretrained_transformer_model_name
# """

# import logging
# import sys
# import traceback
# from datetime import datetime

# from datasets import load_dataset
# from sentence_transformers import SentenceTransformer, losses
# from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
# from sentence_transformers.similarity_functions import SimilarityFunction
# from sentence_transformers.trainer import SentenceTransformerTrainer
# from sentence_transformers.training_args import SentenceTransformerTrainingArguments

# # Set the log level to INFO to get more information
# logging.basicConfig(format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO)

# # You can specify any Hugging Face pre-trained model here, for example, bert-base-uncased, roberta-base, xlm-roberta-base
# model_name = sys.argv[1] if len(sys.argv) > 1 else "distilbert-base-uncased"
# train_batch_size = 16
# num_epochs = 4
# output_dir = (
#     "output/training_stsbenchmark_" + model_name.replace("/", "-") + "-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# )

# # 1. Here we define our SentenceTransformer model. If not already a Sentence Transformer model, it will automatically
# # create one with "mean" pooling.
# model = SentenceTransformer("indobenchmark/indobert-base-p2")

# # 2. Load the STSB dataset: https://huggingface.co/datasets/sentence-transformers/stsb
# train_dataset = load_dataset("sentence-transformers/stsb", split="train")
# eval_dataset = load_dataset("sentence-transformers/stsb", split="validation")
# test_dataset = load_dataset("sentence-transformers/stsb", split="test")
# logging.info(train_dataset)

# # 3. Define our training loss
# # CosineSimilarityLoss (https://sbert.net/docs/package_reference/sentence_transformer/losses.html#cosinesimilarityloss) needs two text columns and one
# # similarity score column (between 0 and 1)
# train_loss = losses.CosineSimilarityLoss(model=model)
# # train_loss = losses.CoSENTLoss(model=model)

# # 4. Define an evaluator for use during training. This is useful to keep track of alongside the evaluation loss.
# dev_evaluator = EmbeddingSimilarityEvaluator(
#     sentences1=eval_dataset["sentence1"],
#     sentences2=eval_dataset["sentence2"],
#     scores=eval_dataset["score"],
#     main_similarity=SimilarityFunction.COSINE,
#     name="sts-dev",
# )

# # 5. Define the training arguments
# args = SentenceTransformerTrainingArguments(
#     # Required parameter:
#     output_dir=output_dir,
#     # Optional training parameters:
#     num_train_epochs=num_epochs,
#     per_device_train_batch_size=train_batch_size,
#     per_device_eval_batch_size=train_batch_size,
#     warmup_ratio=0.1,
#     fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
#     bf16=False,  # Set to True if you have a GPU that supports BF16
#     # Optional tracking/debugging parameters:
#     evaluation_strategy="steps",
#     eval_steps=100,
#     save_strategy="steps",
#     save_steps=100,
#     save_total_limit=2,
#     logging_steps=100,
#     run_name="sts",  # Will be used in W&B if `wandb` is installed
# )

# # 6. Create the trainer & start training
# trainer = SentenceTransformerTrainer(
#     model=model,
#     args=args,
#     train_dataset=train_dataset,
#     eval_dataset=eval_dataset,
#     loss=train_loss,
#     evaluator=dev_evaluator,
# )
# trainer.train()


# # 7. Evaluate the model performance on the STS Benchmark test dataset
# test_evaluator = EmbeddingSimilarityEvaluator(
#     sentences1=test_dataset["sentence1"],
#     sentences2=test_dataset["sentence2"],
#     scores=test_dataset["score"],
#     main_similarity=SimilarityFunction.COSINE,
#     name="sts-test",
# )
# test_evaluator(model)

# # 8. Save the trained & evaluated model locally
# final_output_dir = f"{output_dir}/final"
# model.save(final_output_dir)

# # 9. (Optional) save the model to the Hugging Face Hub!
# # It is recommended to run `huggingface-cli login` to log into your Hugging Face account first
# model_name = model_name if "/" not in model_name else model_name.split("/")[-1]
# try:
#     model.push_to_hub(f"{model_name}-sts")
# except Exception:
#     logging.error(
#         f"Error uploading model to the Hugging Face Hub:\n{traceback.format_exc()}To upload it manually, you can run "
#         f"`huggingface-cli login`, followed by loading the model using `model = SentenceTransformer({final_output_dir!r})` "
#         f"and saving it using `model.push_to_hub('{model_name}-sts')`."
#     )

In [ ]:
# class TokenSimilarityTF:
#     def init(self, max_length, truncation:bool=True, padding:str="max_length"):
#         self.max_length = max_length
#         self.truncation = truncation
#         self.padding = padding

#     def load_pretrained(self, from_pretrained:str="indobenchmark/indobert-base-p1"):
#         self.tokenizer = AutoTokenizer.from_pretrained(from_pretrained)
#         self.model = TFAutoModel.from_pretrained(from_pretrained)

#     def cleaning(self, text:str):
#         text = text.translate(str.maketrans('', '', string.punctuation))
#         text = re.sub(r'\s+', ' ', text).strip()
#         return text

#     def process(self, first_token:str, second_token:str):
#         inputs = self.tokenizer([first_token, second_token],
#                                 max_length=self.max_length,
#                                 truncation=self.truncation,
#                                 padding=self.padding,
#                                 return_tensors='tf')
#         attention = inputs['attention_mask']
#         outputs = self.model(inputs)
#         embeddings = outputs[0]
#         mask = tf.cast(tf.expand_dims(attention, -1), dtype=tf.float32)
#         masked_embeddings = embeddings * mask
#         summed = tf.reduce_sum(masked_embeddings, axis=1)
#         counts = tf.clip_by_value(tf.reduce_sum(mask, axis=1), clip_value_min=1e-9, clip_value_max=tf.float32.max)
#         mean_pooled = summed / counts
#         return mean_pooled.numpy()

#     def predict(self, first_token:str, second_token:str,
#                 return_as_embeddings:bool=False, max_length:int=16,
#                 truncation:bool=True, padding:str="max_length"):
#         self.max_length = max_length
#         self.truncation = truncation
#         self.padding = padding

#         first_token = self.cleaning(first_token)
#         second_token = self.cleaning(second_token)

#         mean_pooled_arr = self.process(first_token, second_token)
#         if return_as_embeddings:
#             return mean_pooled_arr

#         # calculate similarity
#         similarity = cosine_similarity([mean_pooled_arr[0]], [mean_pooled_arr[1]])

#         return similarity